# Data Cleaning

Importing libraries for data cleaning.

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 25000

In [2]:
df = pd.read_csv('../data/retail.csv')
df1 = pd.read_csv('../data/server.csv')

Checking for nulls in both datasets.

In [3]:
df.isnull().sum()

Unnamed: 0         0
title              0
selftext        1097
subreddit          0
created_utc        0
author             0
num_comments       0
score              0
is_self            0
timestamp          0
dtype: int64

In [4]:
df1.isna().sum()

Unnamed: 0         0
title              0
selftext        1202
subreddit          0
created_utc        0
author             0
num_comments       0
score              0
is_self            0
timestamp          0
dtype: int64

Dropping duplicate rows using titles as criteria value. 

In [5]:
df.drop_duplicates(subset = 'title', inplace = True)
df1.drop_duplicates(subset = 'title', inplace = True)

Creating a placeholder where post text has been removed or post text is null.

In [6]:
df['selftext'] = df['selftext'].map(lambda x: "" if x == '[removed]' else x)
df1['selftext'] = df1['selftext'].map(lambda x: "" if x == '[removed]' else x)

In [7]:
df['selftext'] = df['selftext'].fillna('')
df1['selftext'] = df1['selftext'].fillna('')

Creating a "fulltext" column which combines the title and post text. This column will therefore, in the cases where there is no post text, will still provide title text. 

In [8]:
df['fulltext'] = df['title'] + ' ' + df['selftext']
df1['fulltext'] = df1['title'] + ' ' + df1['selftext']

Converting the timestamp column into a pandas datetime column for further filtering and feature creation.

In [9]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

Rewriting the retail and server dataframes with data exclusively after November 1st, 2011. 

In [10]:
df = df[~(df['timestamp'] < '2012-11-01')]

In [11]:
df1 = df1[~(df1['timestamp'] < '2012-11-01')]

Creating temporal features in both datasets that contain the year and month a post was posted, respectively.

In [12]:
df['year'] = pd.DatetimeIndex(df['timestamp']).year
df1['year'] = pd.DatetimeIndex(df1['timestamp']).year

In [13]:
df['month'] = pd.DatetimeIndex(df['timestamp']).month
df1['month'] = pd.DatetimeIndex(df1['timestamp']).month

Dropping unnecessary columns.

In [14]:
df = df.drop(columns = ['created_utc', 'Unnamed: 0', 'title', 'selftext'])
df1 = df1.drop(columns = ['created_utc', 'Unnamed: 0', 'title', 'selftext'])

Checking the shapes of our newly cleaned individual dataframes.

In [15]:
df.shape

(11177, 9)

In [16]:
df1.shape

(11120, 9)

Exporting new dataframes as csv files without indices to the project folder for further exploration.

In [17]:
df.to_csv('retail_clean.csv', index = False)
df1.to_csv('server_clean.csv', index = False)

Concatenating cleaned dataframes, using 'merge' option on the kind arguement to alternate indices, thoroughly shuffling entries from both of the datasets, and then reseting the index to concretize shuffled order. 

In [18]:
df = pd.concat([df, df1]).sort_index(kind='merge')

In [19]:
df.reset_index(drop = True, inplace=True)

Binarizing the target variable so that posts from the "TalesFromYourServer" are represented as 1, while posts from "TalesFromRetail" are represented as 0. 

In [20]:
df['subreddit'] = df['subreddit'].map(lambda x: 1 if x == "TalesFromYourServer" else 0)

Checking on the data and shape of newly merged dataframe, before exporting the dataframe as a csv file to the project folder.

In [21]:
df.head()

,subreddit,author,num_comments,score,is_self,timestamp,fulltext,year,month
0,0,Avacynarchangel,0,1,True,2021-07-11,Closing time.,2021,7
1,1,curlysue6,16,1,True,2021-07-11,Drama with hooking up at work I hooked up with...,2021,7
2,0,DominicB547,5,1,True,2021-07-11,Customers don't Read #1: Soda Water Eggs Chips...,2021,7
3,1,douniee,12,1,True,2021-07-11,Any tips for. Newbie on memorizing a menu?,2021,7
4,0,PM_ME_UR_CATS__,81,1,True,2021-07-11,"No, I'm not giving you another free giveaway. ...",2021,7


In [22]:
df.shape

(22297, 9)

In [23]:
df.to_csv('merged.csv', index = False)